In [10]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Load data
cinema = pd.read_csv("cinema.csv")
cinema['saledate'] = pd.to_datetime(cinema['saledate'])
cinema['year'] = cinema['saledate'].dt.year
cinema['month'] = cinema['saledate'].dt.to_period('M').astype(str)

# Dash app
app = dash.Dash(__name__)
app.title = "Cinema Dashboard"

# Layout
app.layout = html.Div([
    html.H1("🎬 Cinema Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Chọn năm:"),
        dcc.Dropdown(
            id='year-dropdown',
            options=[{"label": y, "value": y} for y in sorted(cinema['year'].unique())],
            value=cinema['year'].min(),
            clearable=False
        )
    ], style={"width": "20%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Chọn quốc gia:"),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{"label": c, "value": c} for c in sorted(cinema['country'].dropna().unique())],
            value=None,
            placeholder="Tất cả quốc gia"
        )
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Nhóm tuổi:"),
        dcc.Dropdown(
            id='agegroup-dropdown',
            options=[{"label": a, "value": a} for a in sorted(cinema['Nhóm tuổi'].dropna().unique())],
            value=None,
            placeholder="Tất cả nhóm tuổi"
        )
    ], style={"width": "25%", "display": "inline-block", "padding": "10px"}),

    html.Div([
        html.Label("Thời gian trong ngày:"),
        dcc.Dropdown(
            id='timeofday-dropdown',
            options=[{"label": t, "value": t} for t in sorted(cinema['Thời gian trong ngày'].dropna().unique())],
            value=None,
            placeholder="Tất cả thời gian"
        )
    ], style={"width": "25%", "display": "inline-block", "padding": "10px"}),

    dcc.Graph(id='revenue-trend'),
    dcc.Graph(id='top-movies'),
    dcc.Graph(id='genre-distribution')
])

# Callbacks
@app.callback(
    [
        Output('revenue-trend', 'figure'),
        Output('top-movies', 'figure'),
        Output('genre-distribution', 'figure')
    ],
    [
        Input('year-dropdown', 'value'),
        Input('country-dropdown', 'value'),
        Input('agegroup-dropdown', 'value'),
        Input('timeofday-dropdown', 'value')
    ]
)
def update_graphs(year, country, agegroup, timeofday):
    filtered = cinema[cinema['year'] == year]
    if country:
        filtered = filtered[filtered['country'] == country]
    if agegroup:
        filtered = filtered[filtered['Nhóm tuổi'] == agegroup]
    if timeofday:
        filtered = filtered[filtered['Thời gian trong ngày'] == timeofday]

    # Revenue trend
    rev_by_month = filtered.groupby('month')['total'].sum().reset_index()
    fig1 = px.line(rev_by_month, x='month', y='total', title='Tổng doanh thu theo tháng')

    # Top movies
    top_movies = filtered.groupby('title')['total'].sum().nlargest(5).reset_index()
    fig2 = px.bar(top_movies, x='title', y='total', title='Top 5 phim doanh thu cao nhất')

    # Genre distribution
    genres = filtered['listed_in'].dropna().str.split(', ').explode()
    genre_counts = genres.value_counts().nlargest(10).reset_index()
    genre_counts.columns = ['Thể loại', 'Số lượt xem']
    fig3 = px.pie(genre_counts, names='Thể loại', values='Số lượt xem', title='Phân bố thể loại phim')

    return fig1, fig2, fig3

# Run server
if __name__ == '__main__':
    app.run(debug=True)
